<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/crim-ca/geoimagenet/blob/master/fine_tuning_resnet.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/crim-ca/geoimagenet/blob/master/fine_tuning_resnet.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/crim-ca/geoimagenet/blob/master/fine_tuning_resnet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Fine Tuning of a ResNet Model

This notebook explains how to load a geoimagenet dataset, fine-tune and package a ResNet-18 model using the [thelper library](https://github.com/plstcharles/thelper). 

First, we need to install the required libraries.

### Environnement Configuration
First, make sure that you have access to a GPU:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator drop-down to GPU.

In [ ]:
%%bash
pip3 --quiet install torch torchvision pillow gitpython lz4 matplotlib numpy pyyaml scikit-learn six tqdm h5py opencv-python  pretrainedmodels albumentations pyyaml
pip3 --quiet install affine geojson shapely pyproj hdf5plugin
pip uninstall thelper
rm -rf ./thelper-src
git clone  https://github.com/plstcharles/thelper.git thelper-src
pip3 install --quiet ./thelper-src

We also need to install an external repo containing new models as well as useful functions.

In [ ]:
%%bash
pip uninstall ginmodelrepo
rm -rf ./ginmodelrepo-src
rm -rf ./ginmodelrepo
git clone https://github.com/crim-ca/gin-model-repo ginmodelrepo-src
pip3 install --quiet ./ginmodelrepo-src

Make sure thelper is imported correctly

In [ ]:
import os
import torch
import torchvision
import thelper
import ginmodelrepo
import json

thelper.utils.init_logger()

We download the GeoImageNet training set:

In [ ]:
# ginmodelrepo.util.maybe_download_and_extract('1-tjv9gznTAM_VEaCeqz9k77fJrvVoTBw','/content/deepglobe_train.zip')
ginmodelrepo.util.maybe_download_and_extract('12_e0dDwFDDIJTNDIBLVzJovs1YhxRhpg','/content/train/gin.zip') 
# !unzip deepglobe_train.zip

In [ ]:
%%bash
cd /content/train/
ls -l *.tif

Install the tensorboard extension:

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard below will track metrics once the training is launched, it must be launched first then hit refresh

In [ ]:
%tensorboard --logdir /content/gin_training/output/gin_training 

## Training Session Configuration

Name your model

In [ ]:
session_name = 'gin-resnet'
dataset_name = 'gin'
DATASET_ROOT = '/content/train'

Open the training set json file in order to prepare the data loader:

In [ ]:
# Opening JSON file 
with open('/content/train/meta.json') as json_file: 
  meta = json.load(json_file)
  dataset= {'path': DATASET_ROOT, ginmodelrepo.util.DATASET_DATA_KEY : meta}

In [ ]:
print(meta['patches'])

We check the available classes in the training set, it is important to specify the right split (train or test)

In [ ]:
taxo_name = 'Land cover'
split = 'train'
class_mapping, all_classes_with_files, all_classes_names, classes_per_taxo, datasets_per_taxo = ginmodelrepo.util.get_dataset_classes(dataset, min_occurence = 5)
class_mapping = datasets_per_taxo[taxo_name]['class_mapping']
print(class_mapping)
inv_class_mapping = {str(v):k for k,v in class_mapping.items()}
if len(all_classes_with_files) == 0:
  raise ValueError("No data for split '{}'".format(split))
dataset[ginmodelrepo.util.DATASET_DATA_KEY][ginmodelrepo.util.DATASET_DATA_PATCH_KEY]= datasets_per_taxo[taxo_name]['data']


Optionally, we can add the background class (everything else outside the masks) or ignore the 0 value in the masks (`dontcare = -1`)

In [ ]:
dontcare = -1
if dontcare is None:
  class_mapping = {'Background': 999, **class_mapping}
n_classes = len(class_mapping.keys())

We configure the classification task in thelper:

In [ ]:
task_config = {
  "type": "thelper.tasks.Classification",
  "params": {
      "class_names": list(dict(class_mapping).keys()),
      "input_key": "data",
      "label_key": "label"
  }
}
task = ginmodelrepo.util.update_class_mapping(class_mapping, task_config)
print(task)

We configure the training set:

In [ ]:

datasets_train= ginmodelrepo.util.adapt_dataset_for_model_task(task, dataset, 'train')
datasets_valid= ginmodelrepo.util.adapt_dataset_for_model_task(task, dataset, 'test')
datasets_config = {
  'gin_train': datasets_train, 
  'gin_valid': datasets_valid,
}

In [ ]:
print(datasets_config)

The base transforms will be applied by the data loader in order to pre-process the images before training. Note that it is important to precise the `target_key` for transforms applicable to the images only (and not the masks):

In [ ]:
base_transforms= [
  {
      "operation": "thelper.transforms.SelectChannels",
      "params": {"channels": [0, 1, 2]},
      "target_key": "data", # here we specify that only the transform is applied on the data and not the mask
  },
  {
      "operation": "thelper.transforms.CenterCrop",
      "params": {"size": 128},
  },
  {
      "operation": "thelper.transforms.NormalizeMinMax",
      "params": {"min": 0.0, "max": 255.0},
      "target_key": "data",
  },
  {
      "operation": "thelper.transforms.NormalizeZeroMeanUnitVar",
      "params": {
          "mean": [0.485, 0.456, 0.406],
          "std": [0.229, 0.224, 0.225]
      },
      "target_key": "data",
  },
  {
      "operation": "thelper.transforms.Transpose",
      "params": {
          "axes": [2,0,1]
      },
      "target_key": "data",
  },
]


Data loader creation, we need to specify the train/validation split:

In [ ]:
loaders_config = {
    "base_transforms": base_transforms,
    "batch_size": 16,
    "workers": 4,
    "pin_memory": True,
    "train_split": {
        "gin_train": 1.0  # 80% of dataset goes to training set
    },
    "valid_split": {
        "gin_valid": 1.0  # remaining 20% goes to validation set
    },
}

data_config = {"datasets": datasets_config, "loaders": loaders_config}  
save_dir = None 
task, train_loader, valid_loader, test_loader = thelper.data.create_loaders(data_config, save_dir=save_dir)

### Minibatch Visualization

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv
print(f"train minibatch count: {len(train_loader)}")
print(inv_class_mapping)
samples = next(iter(train_loader))
print(samples['label'])
print(f"images tensor shape: {samples['data'].shape}")  # BxCxHxW
print(f"labels tensor shape: {samples['label'].shape}")  # Bx1  (une étiquette par image du minibatch)
batch_size = samples['data'].shape[0]
display_batch_size = min(8, batch_size)
fig = plt.figure(figsize=(24, 6))
for r, key in enumerate(['data']):
  for ax_idx in range(display_batch_size):
    ax = fig.add_subplot(1, 8, ax_idx + 1 + r*8)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    class_name = inv_class_mapping[task.class_names[samples['label'][ax_idx].numpy()]] 
    # class_name = samples['label'][ax_idx].numpy()
    # class_name = samples['name']
    ax.set_title(f"{class_name}")
    display = samples[key][ax_idx, ...].numpy()
    if r == 0:
      display = display.transpose((1, 2, 0))  # CxHxW => HxWxC (tel que demandé par matplotlib)
      display = 128 * display + 127  # on inverse la normalisation
    display = cv.normalize(display, dst=None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)
    plt.imshow(display)
plt.show()

### Model Configuration

We are training a ResNet-18 neural network.

In [ ]:
model_config = {
    "type": "torchvision.models.resnet18",
        "params": {"pretrained": True},
}
model = thelper.nn.create_model({"model": model_config}, task)

### Trainer Configuration

Gather the parameters to be optimized/updated in this run. If we are finetuning we will be updating all parameters. However, if we are doing feature extract method, we will only update the parameters that we have just  initialized, i.e. the parameters with `requires_grad = True`.



In [ ]:
base_lr = 0.001

In [ ]:
params_to_update = model.parameters()
n_param = 0
name_block = dict()
lr_blcok = dict()
for l, (name,param) in enumerate(model.named_parameters()):
    n_param += 1
for l, (name,param) in enumerate(model.named_parameters()):
    blcok = name.split('.')[0]
    name_block[l] = name
    lr_blcok[l] = base_lr * 10**(2*(l/n_param-1))

We turn off the model update for half the model

In [ ]:
params_to_update = []
#name_block = set(name_block)
for l, (name,param) in enumerate(model.named_parameters()):
  if l < int(n_param/2):
    param.requires_grad = False # we are freezing the encoder part of the UNet
  else:
    params_to_update.append({
        "params": param,
        "lr": lr_blcok[l],# you can choose to apply different lr value for each layer
    })
  if param.requires_grad == True:
      print("\t",name)


### Training Session Creation

In [ ]:
optimizer_config = {
    "loss": {"type": "torch.nn.CrossEntropyLoss"},
    "optimizer": torch.optim.SGD(params_to_update, lr=base_lr, momentum=0.9, weight_decay=0.001),
}
trainer_config = {
    "epochs": 20,# 20 epochs 
    "tbx": True, # turn on the tensorboard logs
    "monitor": "accuracy",
    "optimization": optimizer_config,
    "metrics": {
        "accuracy": {"type": "thelper.optim.Accuracy"},
    }
}
loaders = (train_loader, valid_loader, test_loader)
trainer = thelper.train.create_trainer(session_name="gin_training",
                                       save_dir="./gin_training",
                                       config={"trainer": trainer_config},
                                       model=model,
                                       task=task,
                                       loaders=loaders)

### Launch the training
Execute that cell and then go back on the Tensorboard cell on top

In [ ]:
outputs = trainer.train()

### Export the Model
Once the training is done we can package and export the best model

In [ ]:
config = thelper.utils.load_checkpoint('/content/gin_training/checkpoints/ckpt.best.pth')
model_config = {
    "type": config["model_type"],
    "params": config["model_params"],
    'weights': config["model"],
}
config["model"] = model_config
config["datasets"] = datasets_config
config["loaders"] = loaders_config
config["trainer"] = {}

thelper.cli.export_model(config, '/content/export/')